# Schematic

A schematic is a graph representation of your circuit.

For complex circuits, a schematic allows you to create symbols and hierarchy levels to represent your circuit.

Having a schematic allows you to also ensure that your layout matches you schematic (design intent).

There are many schematic capturing tools out there:

- Qucs-s: for RF.
- Xschem: for analog circuits.
- Lumerical interconnect: for photonic circuits.

These tools allow you to create schematics with either your mouse or by code.

gdsfactory also allows you to create complex Schematics directly from python with a very simple interface.

In [ ]:
import gdsfactory as gf
import gdsfactory.schematic as gt
import yaml

In [ ]:
c = gf.c.mzi()

# .plot_netlist_graphviz(): This method reads the component's netlist and uses Graphviz (a graph visualization tool) to create a block diagram.
# interactive=False: This parameter specifies that the output should be a static image rather than an interactive plot.
c.plot_netlist_graphviz(interactive=False)

In [ ]:
c.plot_netlist()

Lets create a MZI lattice of 3 elements.

In [ ]:
s = gt.Schematic()
s.add_instance("mzi1", gt.Instance(component=gf.c.mzi(delta_length=10)))
s.add_instance("mzi2", gt.Instance(component=gf.c.mzi(delta_length=100)))
s.add_instance("mzi3", gt.Instance(component=gf.c.mzi(delta_length=200)))
s.add_placement("mzi1", gt.Placement(x=000, y=0))
s.add_placement("mzi2", gt.Placement(x=100, y=100))
s.add_placement("mzi3", gt.Placement(x=200, y=0))

# The connections, or nets, between the components are defined:
# The first net connects the output port o2 of "mzi1" to the input port o1 of "mzi2".
# The second net connects the output port o2 of "mzi2" to the input port o1 of "mzi3".
s.add_net(gt.Net(p1="mzi1,o2", p2="mzi2,o1"))
s.add_net(gt.Net(p1="mzi2,o2", p2="mzi3,o1"))
g = s.plot_graphviz()

You can also create a splitter tree.

In [ ]:
s = gt.Schematic()
s.add_instance("s11", gt.Instance(component=gf.c.mmi1x2()))
s.add_instance("s21", gt.Instance(component=gf.c.mmi1x2()))
s.add_instance("s22", gt.Instance(component=gf.c.mmi1x2()))
s.add_placement("s11", gt.Placement(x=000, y=0))
s.add_placement("s21", gt.Placement(x=100, y=+50))
s.add_placement("s22", gt.Placement(x=100, y=-50))
s.add_net(gt.Net(p1="s11,o2", p2="s21,o1"))
s.add_net(gt.Net(p1="s11,o3", p2="s22,o1"))
g = s.plot_graphviz()

The nice thing is that you can abstract it to have as many levels as you need.

In [ ]:
splitter = gf.components.mmi1x2()
n = 3
dx = 100
dy = 100
s = gt.Schematic()

# This code programmatically creates a binary tree of splitter components for a schematic. Each column in the tree has twice as many splitters as the previous one,
# and the outputs of each splitter are connected to the inputs of two splitters in the next column.

for col in range(n): # Outer Loop: This loop iterates through each column of the tree.
    rows = 2**col

    # Inner Loop (for row in range(rows)): This loop iterates through each row within the current column.
    # The number of rows (rows = 2**col) doubles with each column (1, 2, 4, 8...).
    for row in range(rows):
        
        # For each position in the grid, a splitter component is created and placed. The y position is calculated to keep the column centered vertically.
        s.add_instance(f"s{col}{row}", gt.Instance(component=splitter))
        s.add_placement(
            f"s{col}{row}", gt.Placement(x=col * dx, y=(row - rows / 2) * dy)
        )

        # This if statement ensures that connections are only made for components that are not in the last column.
        # It defines the two nets that connect the outputs (o2, o3) of each splitter to the inputs (o1) of two corresponding splitters in the next column,
        # creating the branching tree structure.
        if col < n - 1:

            # This line connects the first output port (o2) of the current splitter (s{col}{row}) to the input port (o1) of a splitter in the next column (s{col+1}).
            # The specific splitter it connects to is at row 2*row.
            s.add_net(gt.Net(p1=f"s{col}{row},o2", p2=f"s{col+1}{2*row},o1"))

            # This line connects the second output port (o3) of the current splitter to the input port (o1) of another splitter in the next column.
            # This splitter is at row 2*row+1, directly below the previous one.
            s.add_net(gt.Net(p1=f"s{col}{row},o3", p2=f"s{col+1}{2*row+1},o1"))


g = s.plot_graphviz()

In [ ]:
splitter = gf.components.mmi1x2()
n = 5
dx = 100
dy = 100
s = gt.Schematic()

for col in range(n):
    rows = 2**col
    for row in range(rows):
        s.add_instance(f"s{col}{row}", gt.Instance(component=splitter))
        s.add_placement(
            f"s{col}{row}", gt.Placement(x=col * dx, y=(row - rows / 2) * dy)
        )
        if col < n - 1:
            s.add_net(gt.Net(p1=f"s{col}{row},o2", p2=f"s{col+1}{2*row},o1"))
            s.add_net(gt.Net(p1=f"s{col}{row},o3", p2=f"s{col+1}{2*row+1},o1"))


g = s.plot_graphviz()

In [ ]:
dict(s.netlist)

In [ ]:
# s.netlist.model_dump(exclude_none=True)
# This method converts the s.netlist object into a standard Python dictionary.
# The exclude_none=True argument ensures that any keys with None (empty) values are omitted from the dictionary, keeping the output clean.
# The yaml.dump() function takes the dictionary from the previous step and serializes it into a YAML string.
yaml_component = yaml.dump(s.netlist.model_dump(exclude_none=True))
print(yaml_component)

In [ ]:
yaml.dump(s.netlist.model_dump(exclude_none=True), open("schematic.yaml", "w"))

## Python routing

In [ ]:
n = 2**3
splitter = gf.components.splitter_tree(noutputs=n, spacing=(50, 50))
dbr_array = gf.components.array(
    component=gf.c.dbr, rows=n, columns=1, column_pitch=0, row_pitch=3, centered=True
)
s = gt.Schematic()
s.add_instance("s", gt.Instance(component=splitter))
s.add_instance("dbr", gt.Instance(component=dbr_array))
s.add_placement("s", gt.Placement(x=0, y=0))
s.add_placement("dbr", gt.Placement(x=300, y=0))

for i in range(n):
    s.add_net(
        gt.Net(

            # This defines the start point of the connection. It constructs the port name for each of the 8 outputs of the splitter instance named "s".
            p1=f"s,o2_2_{i+1}",

            # This defines the end point of the connection. It constructs the port name for each of the 8 inputs of the DBR array instance named "dbr".
            p2=f"dbr,o1_{i+1}_1",
            name="splitter_to_dbr",

            # Provides instructions for the auto-router that will eventually draw the physical waveguides for these connections.
            # radius=5: The bend radius for the connecting waveguides should be 5 µm.
            # sort_ports=True: Tells the router to reorder the ports to find the shortest, non-crossing paths.
            # cross_section="strip": The connecting waveguides should be standard strip waveguides.
            settings=dict(radius=5, sort_ports=True, cross_section="strip"),
        )
    )

g = s.plot_graphviz()

In [ ]:
dbr_array.pprint_ports()

In [ ]:
splitter.pprint_ports()

In [ ]:
yaml.dump(s.netlist.model_dump(exclude_none=True), open("schematic.yaml", "w"))
yaml_component = yaml.dump(s.netlist.model_dump(exclude_none=True))
print(yaml_component)

In [ ]:
conf = s.netlist.model_dump(exclude_none=True)
yaml_component = yaml.safe_dump(conf)
print(yaml_component)

In [ ]:
c = gf.read.from_yaml(yaml_component)
c